In [2]:
using POMDPs
using Random 
using Parameters
using StaticArrays
using Distributions
using MCTS
using POMDPSimulators

In [3]:
const Vec2 = SVector{2, Float64}
const Vec3 = SVector{3, Float64}

struct mystate
    uavPose::Vec3
    uavHeading::Float64 # radius
    targetPose::Vec2
end

struct myaction
    xy_speed::Float64 # m/s
    z_speed::Float64
    angle::Float64 # radius
end

@with_kw mutable struct UAVMDP <: MDP{mystate, myaction}

    target_velocity::Vec2 = SVector(0.5,0.5)
    target_std::Float64 = 0.1

    r_outScene::Float64 = -70
    r_action::Float64 = -10
    r_reach::Float64 = 100

    discount::Float64 = 0.95
    
    boundary::Int = 100
    landing_r::Int = 1

end

UAVMDP

In [4]:
function POMDPs.isterminal(p::UAVMDP,s::mystate,a::myaction)
    condition1 = (sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2 + s.uavPose[3]^2) < p.landing_r)
    condition2 = s.uavPose[1] > p.boundary || s.uavPose[2]>p.boundary || s.uavPose[3]>p.boundary
    condition = condition1 || condition2
    return condition
end

In [5]:
function POMDPs.generate_s(p::UAVMDP, s::mystate, a::myaction)
    # calculate target state
    target_dt_distance = p.target_velocity
    sensor_noise = rand(Normal(0, p.target_std), 2)
    curr_targ = s.targetPose + target_dt_distance + sensor_noise # next_target_pos(p, s.targetPose)
    # calculate UAV state
    curr_angle = s.uavHeading + a.angle
    xy_dt_distance = a.xy_speed*SVector(cos(curr_angle), sin(curr_angle)) # careful
    z_dt_distance = a.z_speed
    xyz_dt_distance = SVector(xy_dt_distance[1], xy_dt_distance[2], z_dt_distance)
    curr_pos = s.uavPose + xyz_dt_distance
    return mystate(curr_pos, curr_angle, curr_targ)
end

In [6]:
function myreward(p::UAVMDP, s::mystate, a::myaction)
    
    distance_to_target = sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2 + s.uavPose[3]^2)
    
    reward = p.r_action + 1/(distance_to_target+1)
#     out scene punish 
    if s.uavPose[1]>p.boundary || s.uavPose[2]>p.boundary
        reward = reward + p.r_outScene

    end
    
    if distance_to_target < p.landing_r
        reward = reward + p.r_reach

    end
    return reward
end

myreward (generic function with 1 method)

In [7]:
function POMDPs.actions(p::UAVMDP)
    action_space = [myaction(i, j, k) for i=-5.0:5.0,j=-pi:pi,k=-5.0:5.0]
    return action_space
end

In [8]:
function POMDPs.generate_sr(p::UAVMDP, s::mystate, a::myaction, rng::AbstractRNG)
    sp = generate_s(p, s, a)
    r = myreward(p, s, a)
    return sp, r
end

In [9]:
POMDPs.initialstate_distribution(p::UAVMDP) = mystate(SVector(10,10,10),0,SVector(0,0))
POMDPs.discount(p::UAVMDP) = p.discount
solver = MCTSSolver(n_iterations=50, depth=20, exploration_constant=5.0)
UAV = UAVMDP()

UAVMDP
  target_velocity: SArray{Tuple{2},Float64,1,2}
  target_std: Float64 0.1
  r_outScene: Float64 -70.0
  r_action: Float64 -10.0
  r_reach: Float64 100.0
  discount: Float64 0.95
  boundary: Int64 100
  landing_r: Int64 1


In [10]:
policy = solve(solver, UAV)

MCTSPlanner{UAVMDP,mystate,myaction,MCTS.SolvedRolloutEstimator{POMDPPolicies.RandomPolicy{MersenneTwister,UAVMDP,BeliefUpdaters.NothingUpdater},MersenneTwister},MersenneTwister}(MCTSSolver(50, 20, 5.0, MersenneTwister(UInt32[0x480df4a7, 0xa5913337, 0x384344b5, 0x99a6a3f4], Random.DSFMT.DSFMT_state(Int32[1712506924, 1073613510, -1796880516, 1073224875, 500692120, 1073692069, -1293703140, 1073219375, -730076295, 1073210020  …  -590433360, 1073358649, -1937381248, 1072884348, 493971075, -921970395, 62477793, 1892476654, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000000000000000000

In [11]:
###trial
a = action(policy,mystate(SVector(10,10,10),0,SVector(0,0)))

myaction(1.0, 0.8584073464102069, -5.0)

In [35]:
uav_poses = [] # Vector of sequential uav poses
target_poses = []

0-element Array{Any,1}

In [36]:
i=0
terminate = false
reward_sum = 0

current_state = mystate(SVector(10,10,10),0,SVector(0,0)) ##initialize state

while i<100 && !terminate
    current_action = action(policy,current_state)
    current_reward = myreward(UAV, current_state, current_action)
    current_state = generate_s(UAV, current_state, current_action)
    push!(uav_poses, current_state.uavPose)
    push!(target_poses, current_state.targetPose)
    terminate = isterminal(UAV,current_state,current_action)
    reward_sum = reward_sum + current_reward
    i=i+1
    @show reward_sum
end

reward_sum = -9.945416360950874
reward_sum = -19.89272310234088
reward_sum = -29.83983063987627
reward_sum = -39.78513389489164
reward_sum = -49.73066095944209
reward_sum = -59.67806628004031
reward_sum = -69.62767808581205
reward_sum = -79.57757013551459
reward_sum = -89.5271668989641
reward_sum = -99.47770274207927
reward_sum = -109.4301677019853
reward_sum = -119.38445215528156
reward_sum = -129.33965716601023
reward_sum = -139.2956572729514
reward_sum = -149.2529131513967
reward_sum = -159.21170414260223
reward_sum = -169.1718513722309
reward_sum = -179.13310673121896
reward_sum = -189.09539710952893
reward_sum = -199.058793272296
reward_sum = -209.02330860158577
reward_sum = -218.9888773192655
reward_sum = -228.9554782769379
reward_sum = -238.92307157855856
reward_sum = -248.8915742662403
reward_sum = -258.8608823824958
reward_sum = -268.8310173743419
reward_sum = -278.8020619958002
reward_sum = -288.7739636177759
reward_sum = -298.74654256606846
reward_sum = -308.71969907841304
r

In [37]:
uav_poses

100-element Array{Any,1}:
 [10.2837, 10.9589, 10.8584]
 [9.44459, 11.5029, 11.7168]
 [8.6849, 10.8527, 12.5752] 
 [9.09298, 9.93971, 13.4336]
 [10.0842, 10.0721, 14.292] 
 [10.2384, 11.0601, 15.1504]
 [9.33475, 11.4883, 16.0089]
 [8.66781, 10.7432, 16.8673]
 [9.19313, 9.89226, 17.7257]
 [10.1581, 10.1546, 18.5841]
 [10.1802, 11.1544, 19.4425]
 [9.22781, 11.4592, 20.3009]
 [8.66536, 10.6324, 21.1593]
 ⋮                          
 [10.3224, 10.8164, 86.3983]
 [9.59226, 11.4997, 87.2567]
 [8.72993, 10.9933, 88.1151]
 [8.97086, 10.0228, 88.9735]
 [9.96988, 9.97852, 89.8319]
 [10.2957, 10.9239, 90.6903]
 [9.48155, 11.5046, 91.5487]
 [8.69382, 10.8886, 92.4071]
 [9.06108, 9.95845, 93.2655]
 [10.0572, 10.0468, 94.1239]
 [10.255, 11.027, 94.9823]  
 [9.37117, 11.4948, 95.8407]

In [48]:
using Pkg
Pkg.add("DelimitedFiles")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [8bb1440f] + DelimitedFiles 
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [51]:
using DelimitedFiles
writedlm( "uvapos.csv", uav_poses)
writedlm( "targetpos.csv", target_poses)